

# SQLite examples



![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


In [2]:
import sqlite3 as sqlite3
from creating_table import creating_table 

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## SQL Social-Network Query Exercises
### Tables:
- 1. Highschooler ( ID, name, grade )
- 2. Friend ( ID1, ID2 )  The student with ID1 is friends with the student with ID2. Friendship is mutual, so if (1, 5) is in the Friend table, so is (5, 1).
- 3. Likes ( ID1, ID2 )  Liking someone is not necessarily mutual.

### 1.Creating tables:

In [3]:
connection = sqlite3.connect(":memory:")
cursor = connection.cursor()
creating_table("Friend.txt", cursor, connection)
creating_table("Likes.txt", cursor, connection)
creating_table("Highschooler.txt", cursor, connection)

In [4]:
rows = cursor.execute("SELECT * FROM Highschooler;", "").fetchone()
print(rows)
rows = cursor.execute("SELECT * FROM Friend;", "").fetchone()
print(rows)
rows = cursor.execute("SELECT * FROM Likes;", "").fetchone()
print(rows)

(1510, 'Jordan', 9)
(1510, 1381)
(1689, 1709)


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

## 2.  Examples:

- Find the names of all students who are friends with someone named Gabriel. 

In [5]:
sql_str = '''Select name from Highschooler where ID in (select ID2 from Friend where ID1 In  
        (select ID from Highschooler where name like 'Gabriel')) group by ID order by ID;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Alexis',)


- For every student who likes someone 2 or more grades younger than themselves, return that student's name and grade, and the name and grade of the student they like.  

In [6]:
sql_str = ''' Select H1.name, H1.grade, H2.name, H2.Grade  from (Likes join Highschooler as H1 on Likes.ID1 = H1.ID) 
            join Highschooler as H2 on Likes.ID2 = H2.ID  where H1.Grade-H2.Grade>=2 or H2.Grade-H1.Grade>=2;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('John', 12, 'Haley', 10)


- For every pair of students who both like each other, return the name and grade of both students. Include each pair only once, with the two names in alphabetical order. 

In [7]:
sql_str = ''' Select distinct  CASE WHEN H1.name > H2.name THEN H2.name ELSE H1.name END, CASE WHEN H1.name > H2.name THEN H2.grade 
            ELSE H1.grade END, CASE WHEN H1.name > H2.name THEN H1.name ELSE H2.name END, CASE WHEN H1.name > H2.name THEN H1.grade 
            ELSE H2.grade END  from (Likes join Highschooler as H1 on Likes.ID1 = H1.ID) join Highschooler as H2 on Likes.ID2 = H2.ID 
            where Likes.ID1 in (select distinct CASE WHEN L1.ID1 <> L2.ID1 THEN L1.ID1  ELSE L2.ID1 END  from Likes L1 
            inner join Likes L2 on L1.ID1 = L2.ID2 and L1.ID2 = L2.ID1)  order by H1.name, H2.name;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Cassandra', 9, 'Gabriel', 9)


- Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade. 

In [8]:
sql_str = ''' Select name, grade from Highschooler where ID  not in (Select ID1 IDl from Likes union Select ID2 IDl from Likes) order by grade, name;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Jordan', 9)


- For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades. 

In [9]:
sql_str = ''' Select H1.name, H1.grade, H2.name, H2.grade from (Likes join Highschooler as H1 on Likes.ID1 = H1.ID) 
            join Highschooler as H2 on Likes.ID2 = H2.ID  where Likes.ID2 not in (Select NL.ID1 from Likes NL) order by H1.grade, H1.name;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Brittany', 10, 'Kris', 10)


- Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name within each grade. 

In [10]:
sql_str = ''' Select distinct name1, grade1 from (Select H1.name name1, H1.grade grade1, H2.name name2, H2.grade grade2  
            from ((Select ID1 ID_1, ID2 ID_2 from Friend union Select ID2 ID_1, ID1 ID_2 from Friend) join Highschooler as H1 on ID_1 = H1.ID) 
            join Highschooler as H2 on ID_2 = H2.ID  group by ID_1  having count(distinct  H2.grade)  = 1)  where grade1 = grade1  order by grade1, name1;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Jordan', 9)


- For each student A who likes a student B where the two are not friends, find if they have a friend C in common (who can introduce them!). For all such trios, return the name and grade of A, B, and C. 

In [11]:
sql_str = ''' Select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade from ((Select Likes.ID1 LID1, Likes.ID2 LID2 from Friend , Likes where  not (Likes.ID1 = Friend.ID1 and Likes.ID2 = Friend.ID2 or Likes.ID1 = Friend.ID2 and Likes.ID2 =  Friend.ID1)  and Likes.ID1 = Friend.ID1
            Except
            Select Likes.ID1 LID1, Likes.ID2  LID2 from Friend , Likes where  
            (Likes.ID1 = Friend.ID1 and Likes.ID2 = Friend.ID2 or Likes.ID1 = Friend.ID2 and Likes.ID2 =  Friend.ID1)  and Likes.ID1 = Friend.ID1)  
            join  (Select ID1 ID_11, ID2 ID_21 from Friend union Select ID2 ID_11, ID1 ID_21 from Friend) on  ID_11 = LID1 )  
            join  (Select ID1 ID_12, ID2 ID_22 from Friend union Select ID2 ID_12, ID1 ID_22 from Friend)  on  ID_12 = LID2 and ID_21 = ID_22 
            join Highschooler as H1 on LID1 = H1.ID join Highschooler as H2 on LID2 = H2.ID join Highschooler as H3 on ID_21 = H3.ID order by H1.name, H2.name, H3.name;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Andrew', 10, 'Cassandra', 9, 'Gabriel', 9)


- Find the difference between the number of students in the school and the number of different first names. 

In [13]:
sql_str = ''' Select Count(name)-Count(distinct name) from Highschooler;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

(2,)


- Find the name and grade of all students who are liked by more than one other student.

In [14]:
sql_str = ''' Select distinct name, grade from Highschooler where ID in (select  ID2  from Likes  group by ID2 having Count(ID2)>1) ;'''
rows = cursor.execute(sql_str).fetchone()
print(rows)

('Cassandra', 9)


![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)